<a href="https://colab.research.google.com/github/Jovino-dev/MicrosoftLearnToAudio/blob/main/TextProcessor_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio extraction from Microsoft learn module
This notebook allows to convert a full Microsoft learn module to audio

## Download and process all units from module
This cell download the units, process the text, generates the audios and downloads a zip file with the audio contents.
Edit the form values and click the play button at the left

In [ ]:
# @title Execution {"run":"auto","vertical-output":true,"display-mode":"form"}
url = "https://learn.microsoft.com/es-es/training/modules/introduction-power-platform/" # @param {"type":"string"}
language = "es" # @param ["es","en"]
speed = 1.2 # @param {"type":"slider","min":1,"max":2,"step":0.1}
import re
from typing import List
import requests
from bs4 import BeautifulSoup
from gtts import gTTS
from pydub import AudioSegment
import os
import shutil
import zipfile
import sys

# Define the TextProcessor class for cleaning and structuring text.
class TextProcessor:
    def __init__(self):
        # Patterns to clean text
        self.cleanup_patterns = [
            # Remove multiple spaces
            (r'\s+', ' '),
            # Remove multiple newlines
            (r'\n\s*\n\s*\n+', '\n\n'),
            # Clean problematic special characters
            (r'[^\w\s\-.,;:!?¡¿áéíóúñüÁÉÍÓÚÑÜ()[\]"\'/]', ''),
            # Normalize punctuation
            (r'\.{2,}', '.'),
            (r'\?{2,}', '?'),
            (r'!{2,}', '!'),
        ]
        
        # Words/phrases to filter (navigation, UI, etc.)
        self.filter_phrases = [
            'skip to main content', 'breadcrumb navigation', 'table of contents',
            'in this article', 'next steps', 'feedback', 'was this page helpful',
            'submit and view feedback', 'microsoft learn', 'sign in', 'search',
            'browse', 'theme', 'light', 'dark', 'high contrast', 'previous unit',
            'next unit', 'completed', 'check your knowledge', 'knowledge check',
            # User-specific intro phrases
            'leer en ingles', 'agregar', 'agregar al plan', 'logros', 'preguntar a learn', 'completado',
            # User-specific outro phrases
            'comentarios', 'le ha resultado util esta pagina', 
            # Time pattern
            'minutos'
        ]

    def clean_and_structure(self, raw_text):
        """
        Cleans and structures the text for audio conversion
        
        Args:
            raw_text (str): Unprocessed text
            
        Returns:
            str: Clean and structured text
        """
        if not raw_text:
            return ""
        
        # Limpiar texto básico
        text = self._basic_cleanup(raw_text)
        
        # Filtrar contenido no deseado
        text = self._filter_unwanted_content(text)
        
        # Estructurar para audio
        text = self._structure_for_audio(text)
        
        # Normalizar espaciado
        text = self._normalize_spacing(text)
        
        return text.strip()

    def _basic_cleanup(self, text):
        """Basic text cleanup"""
        # Convertir a minúsculas para comparaciones
        text_lower = text.lower()
        
        # Aplicar patrones de limpieza
        for pattern, replacement in self.cleanup_patterns:
            text = re.sub(pattern, replacement, text)
        
        return text

    def _filter_unwanted_content(self, text):
        """Filters unwanted content such as navigation, UI, etc."""
        lines = text.split('\n')
        filtered_lines = []
        
        for line in lines:
            line = line.strip()
            line_lower = line.lower()
            
            # Filtrar líneas muy cortas o vacías
            if len(line) < 3:
                continue
            
            # Filtrar frases específicas
            should_filter = False
            for phrase in self.filter_phrases:
                if phrase in line_lower:
                    should_filter = True
                    break
            
            # Filtrar líneas que parecen navegación o metadatos
            if (line_lower.startswith(('http', 'www.', 'mailto:')) or
                line_lower.endswith(('min', 'sec', 'hr')) or
                re.match(r'^\d+\s*(min|sec|hr|minute|second|hour)', line_lower) or
                re.match(r'^(step \d+|unit \d+|\d+\.)$', line_lower)):
                should_filter = True
            
            if not should_filter:
                filtered_lines.append(line)
        
        return '\n'.join(filtered_lines)

    def _structure_for_audio(self, text):
        """Structures the text to sound more natural when listening"""
        lines = text.split('\n')
        structured_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # Agregar pausas después de títulos/encabezados
            if self._is_heading(line):
                structured_lines.append(f"{line}.")
                structured_lines.append("")  # Línea vacía para pausa
            else:
                # Asegurar que las oraciones terminen con puntuación
                if line and not line[-1] in '.!?':
                    line += '.'
                structured_lines.append(line)
        
        return '\n'.join(structured_lines)

    def _is_heading(self, line):
        """Determines if a line is a heading"""
        # Encabezados suelen ser más cortos y no terminar en puntuación
        return (len(line) < 100 and 
                not line.endswith(('.', '!', '?', ',', ';', ':')) and
                (line.isupper() or line.istitle() or 
                 any(word.isupper() for word in line.split())))

    def _normalize_spacing(self, text):
        """Normalizes text spacing"""
        # Eliminar espacios extra
        text = re.sub(r' +', ' ', text)
        
        # Normalizar saltos de línea
        text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)
        
        # Eliminar espacios al inicio y final de líneas
        lines = [line.strip() for line in text.split('\n')]
        
        return '\n'.join(lines)

    def split_into_chunks(self, text, max_chunk_size=4000):
        """
        Splits the text into chunks for TTS processing, preserving sentence boundaries and natural pauses.
        
        Args:
            text (str): Text to split
            max_chunk_size (int): Maximum size of each chunk
            
        Returns:
            list: List of text chunks
        """
        import re
        # Ensure all sentences end with proper punctuation
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        # Remove empty sentences
        sentences = [s.strip() for s in sentences if s.strip()]
        chunks = []
        current_chunk = ""
        for sentence in sentences:
            # Add a space after each sentence for natural pause
            sentence_with_space = sentence if sentence.endswith(('.', '!', '?')) else sentence + '.'
            sentence_with_space += ' '
            if len(current_chunk) + len(sentence_with_space) > max_chunk_size:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence_with_space
            else:
                current_chunk += sentence_with_space
        if current_chunk:
            chunks.append(current_chunk.strip())
        return chunks


# Function to install necessary dependencies in Colab.
def instalar_dependencias():
    import sys
    !{sys.executable} -m pip install gtts pydub requests beautifulsoup4 pyttsx3
    !apt-get install -y ffmpeg espeak

# Install dependencies.
instalar_dependencias()

# Try importing pyttsx3 (offline TTS).
try:
    import pyttsx3
    offline_tts_available = True
except ImportError:
    offline_tts_available = False

# Function to get unit links from the module URL.
def get_units_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    units = []
    # Find all links with class 'unit-title'.
    for a in soup.find_all('a', class_='unit-title', href=True):
        full_url = requests.compat.urljoin(url, a['href'])
        if full_url not in units:
            units.append(full_url)
    return units

# Function to extract the title of the page.
def extract_title(soup):
    # Try different selectors to find the page title.
    title_elem = soup.select_one('h1[data-bi-name="page-title"], h1.title, h1, .page-title h1, [data-bi-name="page-title"]')
    if title_elem:
        return title_elem.get_text().strip()
    # Fallback to the HTML <title> tag.
    title_tag = soup.find('title')
    if title_tag:
        return title_tag.get_text().strip()
    return "modulo" # Default title if none found.

# Function to convert text to audio using pyttsx3 (offline).
def text_to_audio_offline(text, output_path, language='es', speed=1.0):
    engine = pyttsx3.init()
    # Configure language/voice if possible.
    voices = engine.getProperty('voices')
    selected_voice = None
    for v in voices:
        if language in v.id or language in v.name.lower():
            selected_voice = v.id
            break
    if selected_voice:
        engine.setProperty('voice', selected_voice)
    engine.setProperty('rate', int(200 * speed)) # Set speech speed.
    wav_path = output_path.replace('.mp3', '.wav')
    engine.save_to_file(text, wav_path) # Save audio as WAV.
    engine.runAndWait()
    # Convert WAV to MP3.
    audio = AudioSegment.from_wav(wav_path)
    audio.export(output_path, format='mp3')
    os.remove(wav_path) # Remove the temporary WAV file.

# Main processing function.
def procesar_modulo(url, language, speed):
    units = get_units_links(url)
    if not units:
        print("No se encontraron unidades en el módulo.")
        return None
    print(f"{len(units)} unidades encontradas.")
    # Get module title for output directory name.
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    module_title = extract_title(soup)
    safe_module_title = module_title.replace(' ', '_').replace('/', '_')
    output_dir = f"{safe_module_title}"
    os.makedirs(output_dir, exist_ok=True) # Create output directory.

    # Process each unit.
    for idx, unit_url in enumerate(units, 1):
        print(f"Procesando unidad {idx}: {unit_url}")
        # Limit to processing only the first unit for testing, remove in production.
        # if idx > 1:
        #     break
        unit_resp = requests.get(unit_url)
        unit_soup = BeautifulSoup(unit_resp.content, 'html.parser')
        unit_title = extract_title(unit_soup)
        safe_title = unit_title.replace(' ', '_').replace('/', '_')

        # Extract main content text.
        main_content = unit_soup.select_one('main, .content, [role="main"], .main-content, article, .module-content')
        if not main_content:
            main_content = unit_soup
        text = main_content.get_text(separator='\n')

        # Process and clean the extracted text.
        processor = TextProcessor()
        processed_text = processor.clean_and_structure(text)

        # Convert processed text to audio using gTTS (online).
        audio_path = os.path.join(output_dir, f"unidad_{idx}-{safe_title}.mp3")
        tts = gTTS(text=processed_text, lang=language, slow=False)
        tts.save(audio_path)
        print(f"Audio guardado: {audio_path}")

    return output_dir # Return the path to the output directory.

# Execute the main processing function with input values.
dir_modulo = procesar_modulo(url, language, speed)

if dir_modulo:
    # Define the path for the output zip file.
    zip_path = f"output.zip"
    # Create a zip file containing the audio files.
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        # Walk through the output directory and add files to the zip.
        for root, _, files in os.walk(dir_modulo):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=os.path.join(os.path.basename(dir_modulo), file))
    print(f"ZIP creado: {zip_path}")
    # Import the files module for downloading in Colab.
    from google.colab import files
    # Download the created zip file.
    files.download(zip_path)
else:
    # Print a message if no module was generated.
    print("No se generó ningún módulo para comprimir.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
13 unidades encontradas.
Procesando unidad 1: https://learn.microsoft.com/es-es/training/modules/introduction-power-platform/1-introduction
Audio guardado: output_Describir_el_valor_empresarial_de_Microsoft_Power_Platform/unidad_1-Introducción.mp3
Procesando unidad 2: https://learn.microsoft.com/es-es/training/modules/introduction-power-platform/2-explore-microsoft-power-platform
Audio guardado: output_Describir_el_valor_empresarial_de_Microsoft_Power_Platform/unidad_2-Explorar_Microsoft_Power_Platform.mp3
Procesando unidad 3: https://learn.microsoft.com/es-es/training/modules/introduction-power-platform/3-describe-business-value-power-platform
Audio guardado: output_Describir_el_valor_empresarial_de_Microsoft

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>